### Préférences

Ce module démontre l'usage de l'objet `Decider` implémenté dans le package.

In [12]:
from typing import Tuple
import random
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from projet_sdp.generator import generate_random
from projet_sdp.model import Model
from projet_sdp.decider import Decider

In [2]:
N = 5
E = 3
P = 3
Q = 3

In [3]:
def generate_dataset(number_of_instances: int) -> Tuple[int, int, int]:
    for i in range(number_of_instances):

        plan = generate_random(N=N, E=E, P=P, Q=Q)
        m = Model(plan)

        m.constraint_cant_work_more_than_needed()
        m.constraint_cant_work_on_days_off()
        m.constraint_no_qual_work()
        m.constraint_one_project_per_day()
        m.determine_affecte()
        m.determine_debute()
        m.determine_realise()

        objectives = [m.objective_max_gain, m.objective_min_affecte, m.objective_min_length]
        target_objective = random.choice(objectives)
        target_objective()

        m.update()
        m.optimize(quiet=True)

        yield np.array(m.objective_values())

In [4]:
decider = Decider()

In [5]:
TRAIN_SIZE = 1000
TEST_SIZE = 1000

In [6]:
X_train, X_test = list(generate_dataset(TRAIN_SIZE)), list(generate_dataset(TEST_SIZE))
Y_train, Y_test = list(map(lambda x: decider.decide(x), X_train)), list(map(lambda x: decider.decide(x), X_test))

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-05
(1000, 3) (1000,)
(1000, 3) (1000,)


In [7]:
print(np.count_nonzero(Y_train), "of", TRAIN_SIZE, "pass")

112 of 1000 pass


In [8]:
model = LogisticRegression()
model = model.fit(X_train, Y_train)

In [9]:
Y_pred = model.predict(X_test)

In [10]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00       898
        True       1.00      0.99      1.00       102

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

